# Notebook to play with EMAs

Latest version: 2024-08-1620
Author: MvS

## Description

Similar approach to SMA notebook but using the more volatile exponential moving average (EMA) indicator.

## Result

A vague trading signal (Long/Short) based on Low/High EMAs and a stop-loss estimation.


In [ ]:
import yfinance as yf
import datetime
from math import log2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Standard SMAs
periods = [200, 50]

# parameters of the EMA:
# window length
ema_length = 30
# widening factor
ema_scaler = 1.03

# stock = "FROG"
# stock = "TSLA"
# stock = "NVDA"
# stock = "SAP.TO"
stock = "XOM"


dt_end = datetime.datetime.today()
# Define real-time interval:
#  - assume to display at least the number of sample points of the larger period
#  - this requires double the number of points to create the averaging
#  - plus considering non-trading days - yfinance returns only trading days, howevers
dt_data_start = dt_end - datetime.timedelta(days=max(periods + [ema_length]) * 3)

# Grab sufficient stock data for averaging SMAs
load_df = yf.download(
    stock,
    start=dt_data_start.strftime('%Y-%m-%d'),
    end=dt_end.strftime('%Y-%m-%d'),
    progress=False,
)

### Define dynamic stop-losses based on EMAs

- use worst price quote of each day and calculate an exponential 30-day mean to define a stop-loss for long positions
- reciprocately, the best price quote of each day and calculate an exponential 30-day mean to define a stop-loss for short positions
- both curves are tracking each other with an offset and define a corridor of insignificant price action


In [ ]:
stock_df = load_df.copy()

# Compute the simple moving average (SMA)
for period in periods:
    stock_df[f"SMA_{period:03}"] = stock_df["Close"].rolling(window=period).mean()

# Compute two scaled EMAs based on daily Highs and Lows
stock_df[f"EMA_{ema_length:03}_Long"] = (
    stock_df['Low'].ewm(span=ema_length, adjust=False).mean() / ema_scaler
)  # STOP LOSS LONG
stock_df[f"EMA_{ema_length:03}_Short"] = (
    stock_df['High'].ewm(span=ema_length, adjust=False).mean() * ema_scaler
)  # STOP LOSS SHORT

# Now that we calculated the SMAs and EMAs, we can remove the data points before the actual AOI that we want.
stock_df = stock_df[-max(periods) :].copy()

# Add helpers for time period calculation
dt_start = stock_df.index[0]
stock_df['dt_start'] = dt_start
stock_df['dt_end'] = dt_end

# Add helper to scale signal strength
sig_scale = int(log2(stock_df['SMA_200'][-ema_length:].mean()))
stock_df['sig_scale'] = sig_scale

print(
    f"""Calculating SMAs for {periods} length starting from:
  {dt_start.strftime('%Y-%m-%d')} and ending on {dt_end.strftime('%Y-%m-%d')}
  and containing {stock_df.shape} data points."""
)

print(
    f"""Calculating EMAs for {ema_length} window length
  and containing a {(ema_scaler - 1)*100:4.1f} % scaler."""
)

# Define the columns and their styles
columns = [
    f"SMA_{min(periods):03}",
    f"SMA_{max(periods):03}",
    'High',
    'Low',
    f"EMA_{ema_length:03}_Long",
    f"EMA_{ema_length:03}_Short",
]
colors = ['blue', 'gold', 'darkgreen', 'red', 'forestgreen', 'orangered']
linestyles = ['-', '-', '-', '-', ':', ':']

# Plot each column separately
ax = None
for col, color, style in zip(columns, colors, linestyles):
    ax = stock_df[col].plot(color=color, linestyle=style, ax=ax)

### Using the breakout from the stop-loss corridor as a trading signal

- a trading day where the daily low has cleared the upper bound/short stop-loss can be a long/buy signal
- conversely, a trading day where the daily high has cleared the lower bound/long stop-loss can be a short/sell signal

In [ ]:
# Define the corridors for operation as strong deviations of fast from slow SMA
def valid_signal(x):
    high = f"High"
    low = f"Low"
    long = f"EMA_{ema_length:03}_Long"
    short = f"EMA_{ema_length:03}_Short"

    corridor_scaler = 1.00

    if x[low] > x[short] * corridor_scaler:
        return 1
    elif x[high] < x[long] / corridor_scaler:
        return -1
    else:
        return 0


# don't need loc unless working with view/slice of df
# stock_df.loc[:, 'Valid'] = stock_df.apply(valid_signal, axis=1)
stock_df['Valid'] = stock_df.apply(valid_signal, axis=1)

stock_df['Valid_scaled'] = 2**sig_scale + (stock_df['Valid'] * 2 ** (sig_scale - 2))

# Define the columns and their styles
columns = [
    "High",
    "Low",
    f"EMA_{ema_length:03}_Long",
    f"EMA_{ema_length:03}_Short",
    'Valid_scaled',
]
colors = ['darkgreen', 'red', 'forestgreen', 'orangered', 'grey']
linestyles = ['-', '-', ':', ':', '-']

# Plot each column separately
ax = None
for col, color, style in zip(columns, colors, linestyles):
    ax = stock_df[col].plot(color=color, linestyle=style, ax=ax)

In [ ]:
# Despiking a curve for single outliers

# Shifting curve forward / backward
stock_df['Valid_P_Shift'] = stock_df['Valid'].shift(1)
stock_df['Valid_N_Shift'] = stock_df['Valid'].shift(-1)


# Compare neighbor values: spike has two opposites on either side with same polarity
def despike(x):
    if x['Valid_N_Shift'] == x['Valid_P_Shift'] and x['Valid'] != x['Valid_N_Shift']:
        return x['Valid_N_Shift']
    elif pd.isna(x['Valid_P_Shift']):
        return 0.0
    else:
        return x['Valid']


stock_df['Valid_Despike'] = stock_df.apply(despike, axis=1)

stock_df[['Valid', 'Valid_Despike']].plot(color=['grey', 'red'])

# Replace old with new
stock_df['Valid'] = stock_df['Valid_Despike']
stock_df['Valid_scaled'] = 2**sig_scale + (stock_df['Valid'] * 2 ** (sig_scale - 2))

# clean up
stock_df.drop(
    ['Valid_P_Shift', 'Valid_N_Shift', 'Valid_Despike'],
    axis=1,
    inplace=True,
    errors='ignore',
)

In [ ]:
def get_signal(x):
    if x['Valid'] != x['Shift'] and x['Valid'] != 0:
        return x['Valid']
    else:
        return 0


# Shift forward
stock_df['Shift'] = stock_df['Valid'].shift(1)
# Fill NaN
stock_df['Shift'] = stock_df['Shift'].interpolate(
    method='backfill', limit_direction='backward'
)

# Identify signal onsets
stock_df['On_Signal'] = stock_df.apply(get_signal, axis=1)
stock_df['On_Signal_scaled'] = 2**sig_scale + (
    stock_df['On_Signal'] * 2 ** (sig_scale - 2)
)

# Shift backward
stock_df['Shift'] = stock_df['Valid'].shift(-1)
# Fill NaN
stock_df['Shift'] = stock_df['Shift'].interpolate(
    method='pad', limit_direction='forward'
)

# Identify signal terminations
stock_df['Off_Signal'] = stock_df.apply(get_signal, axis=1)
stock_df['Off_Signal_scaled'] = 2**sig_scale + (
    stock_df['Off_Signal'] * 2 ** (sig_scale - 2)
)

# Define the columns and their styles
columns = [
    "High",
    "Low",
    f"EMA_{ema_length:03}_Long",
    f"EMA_{ema_length:03}_Short",
    'Valid_scaled',
    'On_Signal_scaled',
    'Off_Signal_scaled',
]
colors = ['darkgreen', 'red', 'forestgreen', 'orangered', 'grey', 'red', 'black']
linestyles = ['-', '-', ':', ':', '-', ':', ':']

# Plot each column separately
ax = None
for col, color, style in zip(columns, colors, linestyles):
    ax = stock_df[col].plot(color=color, linestyle=style, ax=ax)

# clean up
stock_df.drop(
    ['Shift'],
    axis=1,
    inplace=True,
    errors='ignore',
)

In [ ]:
if stock_df['On_Signal'].any() != 0:
    print('Some signals found...')

    # Filter rows where either 'On_Signal' or 'Off_Signal' is non-zero
    signals_df = stock_df[
        (stock_df['On_Signal'] != 0) | (stock_df['Off_Signal'] != 0)
    ].copy()

    # Calculate length of signals
    signals_df['Signal_len'] = -signals_df.index.to_series().diff(periods=-1).dt.days
    # Calculate age of signals
    signals_df['Signal_age'] = (
        signals_df['dt_end'] - signals_df.index.to_series()
    ).dt.days
    # Add missing length
    signals_df['Signal_len'].fillna(signals_df['Signal_age'], inplace=True)

    # Calculate gap between signals
    signals_df['Signal_gap'] = signals_df.index.to_series().diff(periods=1).dt.days
    # Calculate ref days of signals
    signals_df['Signal_ref'] = (
        signals_df.index.to_series() - signals_df['dt_start']
    ).dt.days
    # Add missing gap
    signals_df['Signal_gap'].fillna(signals_df['Signal_ref'], inplace=True)

    print(
        signals_df[
            [
                'Valid',
                'On_Signal',
                'Off_Signal',
                'Signal_ref',
                'Signal_len',
                'Signal_gap',
                'Signal_age',
            ]
        ].tail()
    )

In [ ]:
if stock_df['On_Signal'].any() != 0:
    # Add back the signal calculations
    stock_df = pd.concat(
        [
            stock_df,
            signals_df[['Signal_ref', 'Signal_len', 'Signal_gap', 'Signal_age']],
        ],
        axis=1,
    )
    # Fill non-signals
    stock_df.fillna(0, inplace=True)
else:
    stock_df['Signal_ref', 'Signal_len', 'Signal_gap', 'Signal_age'] = [0, 0, 0, 0]

# # Type casting
stock_df = stock_df.astype(
    {
        'Valid': int,
        'Valid_scaled': int,
        'On_Signal': int,
        'On_Signal_scaled': int,
        'Off_Signal': int,
        'Off_Signal_scaled': int,
        'Signal_ref': int,
        'Signal_len': int,
        'Signal_gap': int,
        'Signal_age': int,
        'Valid': int,
    }
)

stock_df.tail()
# stock_df.columns
# stock_df.dtypes

### Plotting the Chart

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(
    data=[
        go.Candlestick(
            x=stock_df.index,
            open=stock_df['Open'],
            high=stock_df['High'],
            low=stock_df['Low'],
            close=stock_df['Close'],
            name=f"{stock}",
        ),
        go.Scatter(
            x=stock_df.index,
            y=stock_df[f"EMA_{ema_length:03}_Long"],
            mode='lines',
            name=f"EMA_{ema_length:03}_Long",
            line=dict(color='#FF5252', width=2, dash='dot'),
        ),
        go.Scatter(
            x=stock_df.index,
            y=stock_df[f"EMA_{ema_length:03}_Short"],
            mode='lines',
            name=f"EMA_{ema_length:03}_Short",
            line=dict(color='#4CAF50', width=2, dash='dot'),
        ),
        go.Scatter(
            x=stock_df.index,
            y=stock_df[f"Valid_scaled"],
            mode='lines',
            name=f"Validity",
            line=dict(color='#000000', width=2, dash='solid'),
        ),
        go.Scatter(
            x=stock_df.index,
            y=stock_df[f"On_Signal_scaled"],
            mode='lines',
            name=f"Signal",
            line=dict(color='#2196F3', width=2, dash='solid'),
        ),
    ]
)

fig.update_yaxes(type='log')

fig.update_layout(
    title='Standard candlesticks and trading signals based on EMAs',
    yaxis_title=f"{stock} Stock",
    xaxis_rangeslider_visible=False,
    width=1200,
    height=800,
)

fig.show()